# First Attempt implementing FC neural network in PyTorch

In [1]:
import torch # God please write code for me

## Problem Description

The aim of the model is to estimate the [IMDB](https://www.imdb.com) score of the a movie, based on:

- Name
- Release Date
- Production Budget
- Genre
- Worldwide Gross
- Run Time

We then try to identify this relationship.

$$\left.
\begin{array}{l}
\text{Film Name} \\
\text{Release Date} \\
\text{Production Budget} \\
\text{Genre} \\
\text{Worldwide Gross} \\
\text{Run Time}
\end{array}
\right\}\Rightarrow\text{IMDB Rating}$$

## Data Source

The data source comes from [here](https://vega.github.io/editor/data/movies.json), licensed under MIT License.

## Preprocessing

For preprocessing, I used a combination of Lua and Python.

### Filter

The original JSON file has more than 3000 films, but many of which has missing information.

The `preprocess.lua` **filters out all entries without any one of the following parameters**:

- Film Name (`Title`)
- Release Date (`Release Date`)
- Production Budget (`Production Budget`)
- Genre (`Main Genre`)
- Worldwide Gross (`Worldwide Gross`)
- Run Time (`Running Time min`)
- IMDB Rating (`IMBD Rating`)

After filtering, **1141 films** survived with **12 different genres**:

```json
["Action","Black Comedy","Comedy","Adventure","Drama","Romantic Comedy","Horror","Thriller/Suspense","Musical","Documentary","Western","Concert/Performance"]
```

The list of movies that survived is stored in `movies_processed.json`.

The list of genres are stored in `genres.json`.

### Encoding

Before feeding the data into the FC neural network, we first need to find a way to encode the data into a list of numbers.

In [2]:
import json, os

with open("genres.json") as f:
    Genres = json.loads(f.read())
with open("movies_processed.json") as f:
    Movies = json.loads(f.read())
print("%d movies found with %d genres." % (len(Movies), len(Genres)))

1140 movies found with 12 genres.


### Input and Output

We'll input all information of a movie into a FC network. But before doing so, proper encoding is necessary.

#### Encoding

In [3]:
import numpy as np
EncodedInput = []

# Titles: stats
maxLen = -1
maxAscii = -1
minAscii = np.Infinity
for i in Movies:
    t = str(i['Title'])
    if len(t) > maxLen:
        maxLen = len(t)
    for char in t:
        if ord(char) < minAscii:
            minAscii = ord(char)
            print("MIN @", char, "({0})".format(ord(char)))
        if ord(char) > maxAscii:
            maxAscii = ord(char)
            print("MAX @", char, "({0})".format(ord(char)))
print('maxLen = {0}\nmaxAscii = {1}\nminAscii = {2}'.format(maxLen, maxAscii, minAscii))

MIN @ B (66)
MAX @ B (66)
MAX @ r (114)
MIN @   (32)
MAX @ w (119)
MAX @ z (122)
MAX @ È (200)
MAX @ ‡ (8225)
maxLen = 58
maxAscii = 8225
minAscii = 32


In the above snippet we found that the maximum length for a title is 58. Several strange symbols are found in addition to regular `(space)` to `z` in ASCII representations. To encode the title, any character with `ord(char) > 122 or ord(char) < 32` will be **clamped** into the range.

The following code implements such encoding.

In [4]:
# Clamping function
def clamp(val, valMin, valMax):
    if val > valMax:
        return valMax, True
    elif val < valMin:
        return valMin, True
    else:
        return val, False

# ASCII encode
TitleEncoded = list(list(clamp(ord(char), 32, 122)[0] for char in str(i["Title"])) for i in Movies)
# Extending to maxLen
TitleEncoded = list(i + [0] * (maxLen - len(i)) for i in TitleEncoded)

# Confirmation
print(Movies[0]["Title"], TitleEncoded[0])

Broken Arrow [66, 114, 111, 107, 101, 110, 32, 65, 114, 114, 111, 119, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


The title in its ASCII form will be given as input to the neural network in **an array of 58 integers** ranging from 32 to 122.

---

Before moving on, define the following normalization function first. It maps an array of data into an array of float between `normMin` and `normMax`.

In [5]:
def normalize(arr, normMin, normMax, inMin = -np.inf, inMax = np.inf):
    if inMax == np.inf and inMin == -np.inf:
        inMin, inMax = min(arr), max(arr)
    return list(map(lambda val: (val - inMin) / (inMax - inMin) * (normMax - normMin) + normMin, arr))

print(TitleEncoded[0])
TitleEncoded = list(normalize(titleEncoded, 0, 1, 0, 122) for titleEncoded in TitleEncoded)
print(Movies[0]["Title"], TitleEncoded[0])
# Add to EncodedInput
EncodedInput.append(TitleEncoded)

[66, 114, 111, 107, 101, 110, 32, 65, 114, 114, 111, 119, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Broken Arrow [0.5409836065573771, 0.9344262295081968, 0.9098360655737705, 0.8770491803278688, 0.8278688524590164, 0.9016393442622951, 0.26229508196721313, 0.5327868852459017, 0.9344262295081968, 0.9344262295081968, 0.9098360655737705, 0.9754098360655737, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Then we'll move on to release dates.

In [6]:
import time

# Example: Jul 25 2008
Timestamps = list(time.mktime(time.strptime(i["Release Date"], "%b %d %Y")) / 100 for i in Movies)
print("Release Date Timestamps ranges from", min(Timestamps), "to", max(Timestamps))

Release Date Timestamps ranges from 3296736.0 to 22074912.0


In the above snippet we found that the time starts with a big integer and has a rather big range. To normalize the time (foundamentally logical because time is consecutive), we set a linear normalization function to clamp all ranges into $[0,\,1]$.

In [7]:
NormalizedTimestamps = normalize(Timestamps, 0, 1)
# Add to EncodedInput
EncodedInput.append(list([i] for i in NormalizedTimestamps))

The timestamps takes **1** float value when given as input to the neural network.

In [8]:
NormalizedProductionBudgets = normalize(list(i["Production Budget"] for i in Movies), 0, 1)
print(NormalizedProductionBudgets[0], Movies[0]["Production Budget"])
EncodedInput.append(list([i] for i in NormalizedProductionBudgets))

0.21664838846239745 65000000


In [9]:
def getGenreId(str):
    return Genres.index(str)
GenreId = list(map(getGenreId, list(i["Major Genre"] for i in Movies)))
print(GenreId[0], Movies[0]["Major Genre"])
EncodedInput.append(list([i / len(Genres)] for i in GenreId))

0 Action


In [10]:
NormalizedWorldwideGross = normalize(list(i["Worldwide Gross"] for i in Movies), 0, 1)
print(NormalizedWorldwideGross[0], Movies[0]["Worldwide Gross"])
EncodedInput.append(list([i] for i in NormalizedWorldwideGross))

0.08049054258816266 148345997


In [11]:
# EncodedInput.append(list([i["Running Time min"]] for i in Movies))
NormalizedRunTime = normalize(list(i["Running Time min"] for i in Movies), 0, 1)
print(NormalizedRunTime[0], Movies[0]["Running Time min"])
EncodedInput.append(list([i] for i in NormalizedRunTime))

0.3522727272727273 108


Before finishing, let's check the final results in `EncodedInput`:

In [12]:
print(len(EncodedInput))
print(list(i[0] for i in EncodedInput), Movies[0])

6
[[0.5409836065573771, 0.9344262295081968, 0.9098360655737705, 0.8770491803278688, 0.8278688524590164, 0.9016393442622951, 0.26229508196721313, 0.5327868852459017, 0.9344262295081968, 0.9344262295081968, 0.9098360655737705, 0.9754098360655737, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.26313610011962824], [0.21664838846239745], [0.0], [0.08049054258816266], [0.3522727272727273]] {'IMDB Votes': 33584, 'Production Budget': 65000000, 'IMDB Rating': 5.8, 'MPAA Rating': 'R', 'Rotten Tomatoes Rating': 55, 'Distributor': '20th Century Fox', 'Director': 'John Woo', 'Creative Type': 'Contemporary Fiction', 'US Gross': 70645997, 'Major Genre': 'Action', 'Worldwide Gross': 148345997, 'Source': 'Original Screenplay', 'Release Date': 'Feb 09 1996', 'Running Time min': 108, 'Title': 'Broken Arrow'}


Now we can generate the encoded input for all movies.

In [13]:
from functools import reduce

def concat(list1, list2):
    list1.extend(list2)
    return list1

EncodedInput = list(reduce(concat, list(i[j] for i in EncodedInput)) for j in range(0, len(Movies)))

In [14]:
pointCount = len(EncodedInput)
print(EncodedInput[0], Movies[0])
inputDimension = len(EncodedInput[0])

[0.5409836065573771, 0.9344262295081968, 0.9098360655737705, 0.8770491803278688, 0.8278688524590164, 0.9016393442622951, 0.26229508196721313, 0.5327868852459017, 0.9344262295081968, 0.9344262295081968, 0.9098360655737705, 0.9754098360655737, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.26313610011962824, 0.21664838846239745, 0.0, 0.08049054258816266, 0.3522727272727273] {'IMDB Votes': 33584, 'Production Budget': 65000000, 'IMDB Rating': 5.8, 'MPAA Rating': 'R', 'Rotten Tomatoes Rating': 55, 'Distributor': '20th Century Fox', 'Director': 'John Woo', 'Creative Type': 'Contemporary Fiction', 'US Gross': 70645997, 'Major Genre': 'Action', 'Worldwide Gross': 148345997, 'Source': 'Original Screenplay', 'Release Date': 'Feb 09 1996', 'Running Time min': 108, 'Title': 'Broken Arrow'}


### Groundtruths

The groundtruth in this case is the IMDB rating of the 1140 movies. Simply compile the groundtruths based on the movies.


In [32]:
GroundTruth = normalize(list(round(i["IMDB Rating"]) for i in Movies), 0, 10, 0, 10)
assert(len(EncodedInput) == len(GroundTruth))

def toOutTensor(result):
    t = [0] * 10
    t[int(result)] = 1
    return t

GroundTruth = list(map(toOutTensor, GroundTruth))
Source = list((EncodedInput[i], GroundTruth[i]) for i in range(len(GroundTruth)))

[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 

## Constructing Neural Network

Now it's time to make a neural network using basic PyTorch functions!

Among the 1140 movies, ID 1-1000 is selected as *training set*, with batch size 25 (40 batches). ID 1001-1100 is *comfirmation set*, with batch size 100 (1 batch). ID 1101-1140 is the *test set*.

In [33]:
# Shuffle sets
def shuffleSource():
    np.random.shuffle(Source)
shuffleSource()

# Configure batch size and count
batchSize = 25
batchCount = 40

def getBatch(i):
    if 25 * i > 1000:
        return False
    else:
        return Source[batchSize * i:batchSize * (i + 1)]

def separateTensorsFromBatch(batch):
    return torch.tensor(list(i[0] for i in batch)), torch.tensor(list(i[1] for i in batch))

In [38]:
dtype = torch.float
device = torch.device("cpu")

# D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
D_in, H, D_out = inputDimension, 250, 10

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-10
for epoch in range(10):
    print("------- EPOCH", epoch,"-------")
    shuffleSource()
    for batchId in range(batchCount):
        trainIn, trainOut = separateTensorsFromBatch(getBatch(batchId))
        # Forward pass: compute predicted y
        h = trainIn.mm(w1)
        h_relu = h.clamp(min=0)
        y_pred = h_relu.mm(w2)
#       print("Output at case 0:", y_pred[0])
#       print("Expected output at case 0:", trainOut[0])
#       Compute and print loss
        loss = (y_pred - trainOut).pow(2).sum().item()
        print("Batch %d : Loss %.2f" % (batchId, loss))

        # Backprop to compute gradients of w1 and w2 with respect to loss
        grad_y_pred = 2.0 * (y_pred - trainOut)
        grad_w2 = h_relu.t().mm(grad_y_pred)
        grad_h_relu = grad_y_pred.mm(w2.t())
        grad_h = grad_h_relu.clone()
        grad_h[h < 0] = 0
        grad_w1 = trainIn.t().mm(grad_h)

        # Update weights using gradient descent
        w1 -= learning_rate * grad_w1
        w2 -= learning_rate * grad_w2


------- EPOCH 0 -------
Batch 0 : Loss 402022.69
Batch 1 : Loss 412950.44
Batch 2 : Loss 426385.56
Batch 3 : Loss 454511.88
Batch 4 : Loss 443004.47
Batch 5 : Loss 420657.97
Batch 6 : Loss 434358.75
Batch 7 : Loss 467683.53
Batch 8 : Loss 358402.16
Batch 9 : Loss 524511.44
Batch 10 : Loss 462919.34
Batch 11 : Loss 528851.31
Batch 12 : Loss 422126.66
Batch 13 : Loss 484731.50
Batch 14 : Loss 415835.50
Batch 15 : Loss 439183.22
Batch 16 : Loss 414344.06
Batch 17 : Loss 445153.88
Batch 18 : Loss 414920.72
Batch 19 : Loss 391676.25
Batch 20 : Loss 354190.78
Batch 21 : Loss 398533.09
Batch 22 : Loss 499589.78
Batch 23 : Loss 419150.59
Batch 24 : Loss 487089.47
Batch 25 : Loss 420115.22
Batch 26 : Loss 478859.72
Batch 27 : Loss 502287.44
Batch 28 : Loss 524879.88
Batch 29 : Loss 407851.06
Batch 30 : Loss 453697.91
Batch 31 : Loss 498618.44
Batch 32 : Loss 395286.06
Batch 33 : Loss 388420.88
Batch 34 : Loss 359758.25
Batch 35 : Loss 394032.78
Batch 36 : Loss 352332.94
Batch 37 : Loss 431834.0

Batch 38 : Loss 416127.56
Batch 39 : Loss 401700.00
------- EPOCH 8 -------
Batch 0 : Loss 464365.59
Batch 1 : Loss 381326.91
Batch 2 : Loss 375974.47
Batch 3 : Loss 414414.22
Batch 4 : Loss 446697.81
Batch 5 : Loss 477851.44
Batch 6 : Loss 439686.25
Batch 7 : Loss 436155.06
Batch 8 : Loss 460213.97
Batch 9 : Loss 441903.59
Batch 10 : Loss 369875.25
Batch 11 : Loss 387578.81
Batch 12 : Loss 403373.34
Batch 13 : Loss 394774.41
Batch 14 : Loss 532736.75
Batch 15 : Loss 521218.34
Batch 16 : Loss 377157.66
Batch 17 : Loss 381897.72
Batch 18 : Loss 454890.03
Batch 19 : Loss 475064.22
Batch 20 : Loss 344048.31
Batch 21 : Loss 485340.03
Batch 22 : Loss 421786.56
Batch 23 : Loss 361949.62
Batch 24 : Loss 476752.81
Batch 25 : Loss 426799.94
Batch 26 : Loss 428489.88
Batch 27 : Loss 503247.84
Batch 28 : Loss 483062.00
Batch 29 : Loss 398722.19
Batch 30 : Loss 371669.00
Batch 31 : Loss 422176.81
Batch 32 : Loss 491350.06
Batch 33 : Loss 442769.97
Batch 34 : Loss 386842.22
Batch 35 : Loss 430970.1